In [1]:
import setup
setup.init_django()

In [2]:
django_pdf="data/django.pdf"

In [8]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.upstash import UpstashVectorStore

In [9]:
reader=documents = SimpleDirectoryReader(input_files=[django_pdf])

In [10]:
import helpers
import openai
openai.api_key=helpers.config("OPENAI_API_KEY",default=None)

In [11]:
url=helpers.config("UPSTASH_VECTOR_URL", default=None)
token=helpers.config("UPSTASH_VECTOR_TOKEN", default=None)
url

'https://upward-humpback-77258-gcp-usc1-vector.upstash.io'

In [ ]:
data=reader.load_data()

In [13]:
import httpx
timeout=httpx.Timeout(600.0, connect=600.0)
client=httpx.Client(timeout=timeout)

In [14]:
from upstash_vector import Index, Vector

upstash_index = Index(url=url, 
              token=token,
             retries=5,
             retry_interval=0.2)
upstash_index._client=client

In [16]:
vector_store = UpstashVectorStore(
    url=url, 
    token=token,
)
vector_store._index = upstash_index

In [18]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [19]:
index = VectorStoreIndex.from_documents(data, storage_context=storage_context, insert_batch_size=500)

In [21]:
index_persist = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [22]:
query_engine=index_persist.as_query_engine()

In [25]:
r=query_engine.query("What are the arguments of embeddings field")
print(r.response)

The arguments of the embeddings field are `srid` and `geom_type`.


In [ ]:
from llama_index.core.response_synthesizers.type import ResponseMode
query_engine = index_persist.as_query_engine()